In [ ]:
#IMPORTS
import numpy as np
# import matplotlib.pyplot as plt
import os
import cv2
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

In [ ]:
# a feature extracting kernel made for increasing accuracy
def extraction (image):
    Kernel  = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    img_k = cv2.filter2D(image,-1, kern)
    img = cv2.cvtColor(img_k, cv2.COLOR_BGR2HSV)
    img[:,:,1] = 255
    img = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)
    return img

In [ ]:
# Loading Datasets

datadir = r"F:/code/detect_data"
# 1: High Rigeness: High rigeness
# 2: Low Rigeness: Low rigeness
# 3: Medium Rigeness: Medium rigeness
# 4: represents a flower or flower (or no apple) : None
Categories = ['1', '2', '3', '4']
training_data = []
test_images = ['Test']
test_data = []
def train ():

    for category in Categories:
        path = os.path.join(datadir, category)
        class_num = Categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                img = cv2.cvtColor(img_array, cv2.COLOR_BGR2HSV)
                img[:,:,1] = 255
                
                img_array = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)
                new_array = cv2.resize(img_array, (100, 100))
                # here i changed flatten to reshaped (1, -1)
                image = np.array(new_array).flatten()
                # image = np.array(image).reshape(1,-1)
                training_data.append([image, class_num])
            except:
                pass

def test ():
    for category in test_images:
        path = os.path.join(datadir, category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))

                img_hsv = cv2.cvtColor(img_array, cv2.COLOR_BGR2HSV)
                img_hsv[:, :, 1] = 255
                img_array = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)
                new_array = cv2.resize(img_array, (100, 100))
                # here i changed flatten to reshaped (1, -1)
                image = np.array(new_array).flatten()
#                 image = np.array(image).reshape(1,-1)

                test_data.append([image])
            except:
                pass


In [ ]:
#TRAINING DATA PREPARATION 
train()
random.shuffle(training_data)
print(len(training_data))
with open('training_data.pkl', 'wb') as f:
    pickle.dump(training_data,f)
print ('Saved Successfully!')


In [ ]:
### SPLITTING DATA INTO ATTRIBUTES AND LABELS ###

with open('training_data.pkl', 'rb') as f:
    training_data = pickle.load(f)

x = []
y = []
for features, label in training_data:
    x.append(features)
    y.append(label)
    


with open('x.pkl', 'wb') as feat:
    pickle.dump(x,feat)
print ('Saved Attributes Successfully!')

with open('y.pkl', 'wb') as lab:
    pickle.dump(y,lab)
print ('Saved labels Successfully!')

In [ ]:
###### LOADING Training dataset for splitting ####

with open('x.pkl', 'rb')as feat:
    X = pickle.load(feat)
with open('y.pkl ', 'rb') as lab:
    Y = pickle.load(lab)




### TRAINING Time bro ###
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.25)


# xtrain = sc.fit_transform(xtrain)
# xtest = sc.transform(xtest)

model = SVC(C=1 , kernel = 'linear', gamma='auto')
# model = CalibratedClassifierCV(model)
model.fit(xtrain, ytrain)

Accuracy = model.score(xtest, ytest)
overfit = model.score(xtrain, ytrain)
predictions = model.predict(xtest)
print ('Accuracy: ', Accuracy)
print ('Overfit: ', overfit)
print(classification_report(ytest, predictions ))


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ytest, predictions)
print (cm)



In [ ]:
#decision tree

from sklearn.tree import DecisionTreeClassifier
from sklearn . ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
bg = BaggingClassifier(SVC(C=1 , kernel = 'linear', gamma='auto', probability = True), max_samples = 1.0 , max_features = 1.0, n_estimators = 30, oob_score = True)

In [ ]:
bg.fit(xtrain , ytrain)

In [ ]:
print(bg.score (xtest, ytest))
bg.predict(xtest)
overfit = bg.score(xtrain, ytrain)
predictions = bg.predict(xtest)
# print ('Accuracy: ', Accuracy)
print ('Overfit: ', overfit)
print(classification_report(ytest, predictions ))


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ytest, predictions)
print (cm)
# print(bg.score (xtrain, ytrain))

In [ ]:
#Assessing the model
clf = cross_val_score(bg, X, Y, cv = 10)
clf

In [ ]:
clf = cross_val_score(model, X, Y, cv = 10)
clf

In [ ]:
# dt = DecisionTreeClassifier(max_depth = 2)
# dt.fit(xtrain, ytrain)
# dt.score(xtrain, ytrain)
rf = RandomForestClassifier(n_estimators = 11)
rf.fit(xtrain, ytrain)
rf.score(xtest, ytest)
# rf.score(xtrain, ytrain

In [ ]:
dt.score(xtest , ytest)


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [1,1.1,1.5, 2,10 ],'gamma': ['scalar', 'auto'],'kernel': ['linear','poly'] }, 
                {'C': [1,1.1,1.5, 2, 10 ],'gamma': [0.1,0.2,0.3,0.4, 0.5, 1 ],'kernel': ['rbf']}, ]

gridsearch = GridSearchCV(estimator = model, param_grid= parameters, scoring = 'accuracy', cv = 10, n_jobs = 2)
gridsearch = gridsearch.fit(xtrain, ytrain)

In [ ]:
print(gridsearch.best_params_)

In [ ]:
### Saving the damn model ###
with open('Banana.pkl', 'wb') as mod:
    pickle.dump(bg, mod)
print (f'Saved pineapple Successfully!')

In [ ]:
### Testing images to confirm stuff, i.e importing the image directly ###
img = cv2.imread('1.jpg')
img = cv2.resize(img, (100,100))
img = np.array(img).reshape(1,-1)

cv2.imshow('test',img)
cv2.waitKey(0)
cv2.destroyAllWindows()